In [58]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

In [59]:
# Load and normalizde the data

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batchSize = 5
validSize = 0.2 # use 20% of train set as validation

trainValidSet = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testSet = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainSet, validSet = torch.utils.data.random_split(trainValidSet, [int(len(trainValidSet)*(1-validSize)), int(len(trainValidSet)*validSize)])

trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=batchSize, shuffle=True)
validLoader = torch.utils.data.DataLoader(validSet, batch_size=batchSize, shuffle=True)
testLoader = torch.utils.data.DataLoader(testSet, batch_size=batchSize, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [60]:
# Writer for tensorBoard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [61]:
len(trainLoader)*batchSize, len(validLoader)*batchSize, len(testLoader)*batchSize, next(iter(testLoader))[0][0].shape

(8000, 2000, 2000, torch.Size([3, 32, 32]))

In [62]:
# Define the network class

class ConvNet(nn.Module):
    def __init__(self, activation):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 100)
        self.fc2 = nn.Linear(100, 10)
        self.activation = activation

    def forward(self, x):
        x = self.pool(self.activation(self.conv1(x)))
        x = self.pool(self.activation(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return x


In [63]:
def trainNetwork(network, optimizer, lossFunction, epochs, writer):
    network.train()
    for epoch in range(epochs):

        ### TRAINING ###
        trainLoss = 0
        for batch_nr, (images, labels) in enumerate(trainLoader):

            # Predict
            predictions = network(images)

            # Get loss and backpropogate
            loss = lossFunction(predictions, labels)
            loss.backward() 

            # Optimize parameters (weights and biases) and remove gradients after
            optimizer.step() 
            optimizer.zero_grad()

            # Save loss for whole epoch
            trainLoss += loss.item()

        trainLoss /= len(trainLoader)
        writer.add_scalar("Loss/train", trainLoss, epoch)

        ### VALIDATION ###
        validLoss = 0
        for batch_nr, (images, labels) in enumerate(validLoader):

            # Predict
            predictions = network(images)

            # Get loss
            loss = lossFunction(predictions, labels)

            # Save loss for whole epoch
            validLoss += loss.item()

        validLoss /= len(validLoader)
        writer.add_scalar("Loss/valid", validLoss, epoch)

        # Print reuslt of epoch
        print(f'Epoch [{epoch+1}/{epochs}] \t Training Loss: {trainLoss} \t Validation Loss: {validLoss}')

    writer.flush()

In [64]:
def testNetwork(network):
    correct_predictions = 0

    ### TESTING ###
    with torch.no_grad(): 
        for batch_nr, (images, labels) in enumerate(testLoader):
            # Get predictions, convert to number and get the amount of correct predicitons
            predictions = network(images)
            _, predictions = torch.max(predictions, 1) 
            correct_predictions += (predictions == labels).sum().item() 
        
    print(f"Accuracy: {100 * correct_predictions / len(testSet)}%")

In [65]:
learningRate = 0.0001
epochs = 5

In [66]:
networkLeakyRelu = ConvNet(F.leaky_relu)

optimizerSGD = torch.optim.SGD(networkLeakyRelu.parameters(), lr=learningRate)
lossFunction = nn.CrossEntropyLoss()

In [67]:
trainNetwork(networkLeakyRelu, optimizerSGD, lossFunction, epochs, writer)

KeyboardInterrupt: 

In [ ]:
testNetwork(networkLeakyRelu)

Accuracy: 49.89%


In [ ]:
optimizerAdam = torch.optim.Adam(networkLeakyRelu.parameters(), lr=learningRate)
trainNetwork(networkLeakyRelu, optimizerAdam, lossFunction, epochs, writer)

Epoch [1/5] 	 Training Loss: 1.440850383322686 	 Validation Loss: 1.3030928874537349
Epoch [2/5] 	 Training Loss: 1.2450615693219007 	 Validation Loss: 1.220072646856308
Epoch [3/5] 	 Training Loss: 1.1355091712088323 	 Validation Loss: 1.1857325531765819
Epoch [4/5] 	 Training Loss: 1.0489152443706988 	 Validation Loss: 1.152480256980285
Epoch [5/5] 	 Training Loss: 0.9879191253466997 	 Validation Loss: 1.2183631848841905


In [ ]:
testNetwork(networkLeakyRelu)

Accuracy: 59.77%


In [ ]:
networkTanh = ConvNet(torch.tanh)
trainNetwork(networkTanh, optimizerAdam, lossFunction, epochs, writer)

Epoch [1/5] 	 Training Loss: 2.307291491150856 	 Validation Loss: 2.306044224023819
Epoch [2/5] 	 Training Loss: 2.3072914925813675 	 Validation Loss: 2.3060442237854004
Epoch [3/5] 	 Training Loss: 2.3072914919257164 	 Validation Loss: 2.306044226169586
Epoch [4/5] 	 Training Loss: 2.307291492551565 	 Validation Loss: 2.3060442258119584
Epoch [5/5] 	 Training Loss: 2.307291490882635 	 Validation Loss: 2.306044228553772


In [ ]:
testNetwork(networkTanh)

Accuracy: 10.82%
